In [6]:
import cv2
import dlib
from PIL import Image
import imutils
import numpy as np
from sklearn.decomposition import PCA
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

In [9]:
# Displayinh the number of GPUs available
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)

Num GPUs Available:  0


In [ ]:
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task

In [11]:
# load a test image, resize it, and convert it to grayscale
def load_image(path, show=False, gray_show=False):
    test_image = cv2.imread(path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    test_image = imutils.resize(test_image, width=500)
    gray = cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

    if show:
        plt.imshow(test_image)
        plt.show()
        if gray_show:
            plt.imshow(gray, cmap='gray')
            plt.show()
    return test_image, gray

def detect_faces(face_detector, gray, img, scale_factor, frame_num, show_multi_faces = False):
    detections = haar_cascade.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=5, minSize=(150, 150), flags=cv2.CASCADE_SCALE_IMAGE)
    #Checking if there are more than 1 detected faces
    if len(detections) > 1:
        print('multiple faces', frame_num)
        if detections[0][2] > detections[1][2]:
            face = detections[0]
            other = detections[1]
        else:
            face = detections[1]
            other = detections[0]
        if show_multi_faces:
            print('largest and chosen face')
            fX, fY, fW, fH = face[0],face[1], face[2], face[3]
            plt.imshow(gray[fY:fY + fH, fX:fX + fW], cmap='gray')
            plt.show()
            print('not chosen face')
            fX, fY, fW, fH = other[0], other[1], other[2], other[3]
            plt.imshow(gray[fY:fY + fH, fX:fX + fW], cmap='gray')
            plt.show()
    elif len(detections) == 1:
        face = detections[0]
    # if no face detected
    elif len(detections) == 0:
        print('No face detected', frame_num)
        return False
    fX, fY, fW, fH = face[0], face[1], face[2], face[3]
    face_img = Image.fromarray(img[fY:fY + fH, fX:fX + fW])
    face_img = face_img.resize((224, 224))
    #face_img.show()
    return face_img

In [12]:
# haar Cascade path
cascade_path = '../models/haarcascade_frontalface_default.xml'
# create haar cascade
haar_cascade = cv2.CascadeClassifier(cascade_path)
face_detector = 'haar'

frame_start = 0
frame_end = 2000
frames = range(frame_start, frame_end + 1)

scale_factor = 1.05

folder_path = '../test_frames/frame'

In [13]:
def get_embeddings(frames, folder_path, face_detector, model, scale_factor, show=False, show_multi_faces=False):
    no_face_count = 0
    embeddings = []
    for frame in frames:
        if frame % 1000 == 0:
            print(frame)
        path = folder_path + str(frame) + '.jpg'
        test_img, gray = load_image(path, show=False)
        face = detect_faces(face_detector, gray, test_img, scale_factor, frame, show_multi_faces=False)

        if not face:
            no_face_count += 1
            continue

        face_array = np.asarray(face)
        pixels = face_array.astype('float32')
        samples = np.expand_dims(pixels, axis=0)
        samples = preprocess_input(samples, version=2)
        embedding = model.predict(samples)
        embeddings.append(embedding[0])
    print(no_face_count)
    return embeddings

embeddings = get_embeddings(frames, folder_path, face_detector, model, scale_factor)

0
1000
2000
0


In [14]:
# Extracts average embedding in a segment from given embeddigs
def embeddings_segment(embeddings, video_len, segment_len):
    # The amount of frames at the end that are not taken into account
    rest = video_len % segment_len
    num_frames = video_len - rest
    avg_embeddings = []
    acum_embeddings = np.zeros(2048)
    # a blink is counted to a segment,when the blink starts in that segment
    for frame in range(num_frames + 1):
        if frame % 1000 == 0:
            print(frame)
        acum_embeddings = acum_embeddings + np.array(embeddings[frame])
        # only happens at the end of a segment
        if frame % segment_len == 0 and frame != 0:
            avg_embeddings.append(acum_embeddings / segment_len)
            acum_embeddings = np.zeros(2048)
            print('New segment: ', frame)
    print(len(avg_embeddings))       
    print(frame)
    return avg_embeddings

avg_embeddings = embeddings_segment(embeddings, len(embeddings), 100)

0
New segment:  100
New segment:  200
New segment:  300
New segment:  400
New segment:  500
New segment:  600
New segment:  700
New segment:  800
New segment:  900
1000
New segment:  1000
New segment:  1100
New segment:  1200
New segment:  1300
New segment:  1400
New segment:  1500
New segment:  1600
New segment:  1700
New segment:  1800
New segment:  1900
2000
New segment:  2000
20
2000


In [17]:
pca = PCA(n_components=2)
pri_comps = pca.fit_transform(avg_embeddings)

print(pri_comps)

[[-12.69764201   0.75867569]
 [ -3.49717356   4.75428519]
 [  0.03350947   3.94221052]
 [ -7.18227163  -0.71232617]
 [ -3.73438158   3.11842092]
 [ -7.89352724   3.73389222]
 [ -2.52443963   7.75185436]
 [  4.68409637  11.71814638]
 [  5.72731744   9.09796273]
 [  1.59666847   7.76002795]
 [ 10.31069058   3.19196255]
 [  4.29602813   4.08033612]
 [ -3.05994379  -0.4031873 ]
 [ -1.53785661 -10.87733395]
 [  2.81343141 -11.69787166]
 [ -5.56690645 -10.63370449]
 [ -4.69013006  -8.21622264]
 [ -2.4298682   -6.52086002]
 [  3.93715916  -2.6044864 ]
 [ 21.41523973  -8.24178202]]
